In [1]:
cd ..

/Users/bdboy/Desktop/Projects/Music-Generation


In [2]:
from melGAN import Generator, Discriminator
from dataset import AudioDataset

In [3]:
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
AUDIO_DIR = "/Users/bdboy/Desktop/Projects/Music-Generation/data/drums/test"
SAMPLE_RATE = 16000
NUM_SAMPLES = 8000

if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Using device {DEVICE}")

TRANSFORM = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64
)

drums = AudioDataset(AUDIO_DIR, TRANSFORM, SAMPLE_RATE, NUM_SAMPLES, DEVICE)
dataloader = DataLoader(drums, batch_size=16, shuffle=True)

print(f"There are {len(drums)} samples in the dataset.")
signal = drums[0]
print(f"Shape of signal: {signal.shape}")

Using device cpu
There are 224 samples in the dataset.
Shape of signal: torch.Size([1, 64, 16])


In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [7]:
netG = Generator().to(DEVICE)
netD = Discriminator().to(DEVICE)

netG.apply(weights_init)
netD.apply(weights_init)

print(netG)
print(netD)

Generator(
  (activ): SELU()
  (conv1): ConvTranspose2d(50, 256, kernel_size=(3, 2), stride=(2, 2))
  (conv2): ConvTranspose2d(256, 128, kernel_size=(3, 2), stride=(2, 2))
  (conv3): ConvTranspose2d(128, 32, kernel_size=(3, 2), stride=(2, 2))
  (conv4): ConvTranspose2d(32, 8, kernel_size=(3, 2), stride=(2, 2), output_padding=(1, 1))
  (conv5): ConvTranspose2d(8, 1, kernel_size=(3, 2), stride=(2, 1), padding=(1, 1), output_padding=(1, 0))
)
Discriminator(
  (activ): SELU()
  (sigmoid): Sigmoid()
  (conv1): Conv2d(1, 4, kernel_size=(3, 2), stride=(2, 1), padding=(1, 1))
  (conv2): Conv2d(4, 16, kernel_size=(3, 2), stride=(2, 2))
  (conv3): Conv2d(16, 32, kernel_size=(3, 2), stride=(2, 2))
  (conv4): Conv2d(32, 64, kernel_size=(3, 2), stride=(2, 2))
  (fc1): Linear(in_features=384, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [8]:
lr = 0.0002
beta1 = 0.5

loss = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [21]:
num_epochs = 200
real_label = 1
fake_label = 0


img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):


        netD.zero_grad()

        real_cpu = data.to(DEVICE)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=DEVICE)
        output = netD(real_cpu).view(-1)
        errD_real = loss(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, 50, 1, 1, device=DEVICE)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach()).view(-1)
        errD_fake = loss(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake

        optimizerD.step()


        
        netG.zero_grad()

        label.fill_(real_label)
        output = netD(fake).view(-1)
        errG = loss(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()

        optimizerG.step()

        # Output training stats
        if epoch%10==0 and i==13:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        G_losses.append(errG.item())
        D_losses.append(errD.item())

Starting Training Loop...
[0/200][13/14]	Loss_D: 0.0402	Loss_G: 4.0617	D(x): 0.9816	D(G(z)): 0.0182 / 0.0231
[10/200][13/14]	Loss_D: 0.0545	Loss_G: 3.7670	D(x): 0.9995	D(G(z)): 0.0490 / 0.0350
[20/200][13/14]	Loss_D: 0.1194	Loss_G: 3.2623	D(x): 0.9554	D(G(z)): 0.0419 / 0.0549
[30/200][13/14]	Loss_D: 0.0530	Loss_G: 4.1653	D(x): 0.9722	D(G(z)): 0.0159 / 0.0423
[40/200][13/14]	Loss_D: 0.2329	Loss_G: 4.0281	D(x): 0.9206	D(G(z)): 0.0828 / 0.0303
[50/200][13/14]	Loss_D: 0.1259	Loss_G: 3.4511	D(x): 0.9560	D(G(z)): 0.0474 / 0.0545
[60/200][13/14]	Loss_D: 0.3181	Loss_G: 2.4236	D(x): 0.8777	D(G(z)): 0.0618 / 0.1126
[70/200][13/14]	Loss_D: 0.0075	Loss_G: 6.3225	D(x): 0.9991	D(G(z)): 0.0065 / 0.0047
[80/200][13/14]	Loss_D: 0.3513	Loss_G: 2.8945	D(x): 0.8708	D(G(z)): 0.0186 / 0.0793
[90/200][13/14]	Loss_D: 0.0792	Loss_G: 3.5109	D(x): 1.0000	D(G(z)): 0.0729 / 0.0509
[100/200][13/14]	Loss_D: 0.2066	Loss_G: 4.8720	D(x): 0.9197	D(G(z)): 0.0061 / 0.0167
[110/200][13/14]	Loss_D: 0.0129	Loss_G: 6.8598	D(x